In [3]:

import requests
import numpy as np
import pandas as pd
import time
from bs4 import BeautifulSoup
from fake_useragent import UserAgent

In [62]:
page_link = 'https://hh.ru/search/resume?logic=normal&pos=full_text&exp_period=all_time&filter_exp_period=all_time&relocation=living_or_relocation&gender=unknown&order_by=relevance&search_period=0&items_on_page=100&no_magic=true&hhtmFrom=resume_search_form&area=113&label=only_with_salary&professional_role=96&text=%D1%80%D0%B0%D0%B7%D1%80%D0%B0%D0%B1%D0%BE%D1%82%D1%87%D0%B8%D0%BA'
UserAgent().chrome
response = requests.get(page_link, headers={'User-Agent': UserAgent().chrome})

response

<Response [200]>

In [63]:
html = response.content
soup = BeautifulSoup(html,'html.parser')

В первой строчке получаем столбцы(характеристики) для нашего датафрейма. Все они лежат в теге div с классом resume-block. Далее для каждой характеристики кандидата получаем текст, который он написал в эту категорию, и добавляем его в список

In [8]:
def get_information_about_candidate1(soup):
    obj = soup.findAll('div', attrs = {'data-qa':'resume-serp__resume'})
    employers_data_frame = pd.DataFrame(columns = ['опыт работы', 'регион', 'пол', 'возраст', 'образование', 'вуз', 'язык', 'занятость', 'желаемая з/п', 'Последнее место работы'])
    for resume in obj:
        employer_id = resume.a.attrs['href']
        employer_id = 'https://hh.ru' + employer_id
        employer_response = requests.get(employer_id, headers={'User-Agent': UserAgent().chrome})
        employer_html = employer_response.content
        employer_soup = BeautifulSoup(employer_html,'html.parser')
        employers_data_frame.loc[len(employers_data_frame)] = None
        try:
            employers_data_frame.iat[-1, 3] = employer_soup.find('span', attrs = {'data-qa' : "resume-personal-age"}).text
        except:
            employers_data_frame.iat[-1, 3] = employer_soup.find('span', attrs = {'data-qa' : "resume-personal-age"})
        try:
            employers_data_frame.iat[-1, 2] = employer_soup.find('span', attrs = {'data-qa' : "resume-personal-gender"}).text
        except:
            employers_data_frame.iat[-1, 2] = employer_soup.find('span', attrs = {'data-qa' : "resume-personal-gender"})
        try:
            ebanutsa = employer_soup.findAll('div', attrs = {'class' : 'resume-block-container'})
            for i in range(len(ebanutsa)):
                if ebanutsa[i].p == None:
                    pass
                else:
                    employers_data_frame.iat[-1, 7] = ebanutsa[i].p.text
                    break
        except:
            pass
        try:
            employers_data_frame.iat[-1, 8] = employer_soup.find('span', attrs = {'class' : 'resume-block__salary'}).text
        except:
            employers_data_frame.iat[-1, 8] = employer_soup.find('span', attrs = {'class' : 'resume-block__salary'})
        try:
            employers_data_frame.iat[-1, 0] = employer_soup.find('span', attrs = {'class' : 'resume-block__title-text resume-block__title-text_sub'}).span.text
        except:
            pass
        try:
            employers_data_frame.iat[-1, 1] = employer_soup.find('span', attrs = {'data-qa' : 'resume-personal-address'}).text
        except:
            employers_data_frame.iat[-1, 1] = employer_soup.find('span', attrs = {'data-qa' : 'resume-personal-address'})
        try:
            employers_data_frame.iat[-1, 4] = employer_soup.find('div', attrs = {'data-qa' : 'resume-block-education'}).span.text
        except:
            pass
        try:
            employers_data_frame.iat[-1, 5] = employer_soup.find('div', attrs = {'data-qa' : 'resume-block-education-name'}).text
        except:
            pass
        try:
            employers_data_frame.iat[-1, 9] = employer_soup.find('div', attrs = {'class' : "bloko-column bloko-column_xs-4 bloko-column_s-2 bloko-column_m-2 bloko-column_l-2"}).text
        except:
            pass
        try:
            ebanutsax2 = employer_soup.findAll('p', attrs = {'data-qa' : 'resume-block-language-item'})
            employers_data_frame.iat[-1, 6] = ' '.join([i.text for i in ebanutsax2])
        except:
            pass

    return employers_data_frame

In [60]:
get_information_about_candidate1(soup)

,опыт работы,регион,пол,возраст,образование,вуз,язык,занятость,желаемая з/п,Последнее место работы
0,17 лет,Москва,Мужчина,39 лет,Высшее образование,Томский политехнический университет,Английский — C2 — В совершенстве Русский — B2 ...,Занятость: полная занятость,NaN,Апрель 2008 — по настоящее время16 лет 2 месяца
1,18 лет,Москва,Мужчина,37 лет,Образование,NaN,Русский — Родной,Занятость: полная занятость,NaN,Октябрь 2005 — по настоящее время18 лет 8 месяцев
2,3 года,Москва,Мужчина,32 года,Высшее образование,Московский государственный технический универс...,Русский — Родной Английский — B1 — Средний,Занятость: полная занятость,NaN,Май 2021 — по настоящее время3 года 1 месяц
3,25 years,Tomsk,Male,56 years,Higher education,Томский Институт Автоматизированных Систем Упр...,Russian — Native English — B1 — Intermediate,"Employment: full time, part time, project work",50 000 ₽ in hand,September 1998 — currently25 years 9 months
4,3 года,Москва,Мужчина,27 лет,Высшее образование,ГГТУ,Русский — Родной Английский — B2 — Средне-прод...,Занятость: полная занятость,None,Апрель 2023 — по настоящее время1 год 2 месяца
5,2 года,Москва,Мужчина,27 лет,Высшее образование (Магистр),Российский государственный университет нефти и...,Русский — Родной Английский — B1 — Средний,Занятость: полная занятость,None,Октябрь 2021 — по настоящее время2 года 8 месяцев
6,3 года,Москва,Мужчина,32 года,Высшее образование (Бакалавр),Российский государственный социальный универси...,Русский — Родной Английский — B1 — Средний,"Занятость: полная занятость, частичная занятость",None,Ноябрь 2020 — Ноябрь 20233 года 1 месяц
7,15 лет,Москва,Мужчина,54 года,Образование,NaN,Английский — B2 — Средне-продвинутый,Занятость: полная занятость,None,Сентябрь 2008 — по настоящее время15 лет 9 мес...
8,17 лет,Москва,Мужчина,37 лет,Образование,NaN,Английский — C2 — В совершенстве,Занятость: полная занятость,70 000 ₽ на руки,Май 2007 — по настоящее время17 лет 1 месяц
9,5 лет,Москва,Мужчина,22 года,Высшее образование,"Московский политехнический университет, Москва",Русский — Родной Английский — B2 — Средне-прод...,Занятость: полная занятость,None,Февраль 2023 — по настоящее время1 год 4 месяца


In [9]:
employers_data_frame = pd.DataFrame(columns = ['опыт работы', 'регион', 'пол', 'возраст', 'образование', 'вуз', 'язык', 'занятость', 'желаемая з/п', 'Последнее место работы'])
for i in range(50):
    page_link = 'https://hh.ru/search/resume?logic=normal&pos=full_text&exp_period=all_time&filter_exp_period=all_time&relocation=living_or_relocation&gender=unknown&order_by=relevance&search_period=0&items_on_page=100&no_magic=true&hhtmFrom=resume_search_form&area=113&label=only_with_salary&professional_role=96&text=%D1%80%D0%B0%D0%B7%D1%80%D0%B0%D0%B1%D0%BE%D1%82%D1%87%D0%B8%D0%BA'+'&page={}'.format(i)
    UserAgent().chrome
    response = requests.get(page_link, headers={'User-Agent': UserAgent().chrome})
    html = response.content
    soup = BeautifulSoup(html,'html.parser')
    employers_data_frame = pd.concat([employers_data_frame, get_information_about_candidate1(soup)], ignore_index = True)

In [11]:
employers_data_frame

,опыт работы,регион,пол,возраст,образование,вуз,язык,занятость,желаемая з/п,Последнее место работы
0,24 года,Санкт-Петербург,Мужчина,45 лет,Неоконченное высшее образование,Академия Культуры,Русский — Родной Английский — B1 — Средний,Занятость: полная занятость,100 000 ₽ на руки,Январь 2000 — по настоящее время24 года 5 месяцев
1,26 лет,Самара,Мужчина,49 лет,Высшее образование,Самарский государственный аэрокосмический унив...,Русский — Родной Английский — B2 — Средне-прод...,Занятость: полная занятость,1 599 € на руки,Январь 2006 — по настоящее время18 лет 5 месяцев
2,2 года,Екатеринбург,Мужчина,23 года,Неоконченное высшее образование,Уральский федеральный университет имени первог...,Русский — Родной Английский — B1 — Средний,Занятость: полная занятость,200 000 ₽ на руки,Июль 2021 — по настоящее время2 года 11 месяцев
3,25 years,Tomsk,Male,56 years,Higher education,Томский Институт Автоматизированных Систем Упр...,Russian — Native English — B1 — Intermediate,"Employment: full time, part time, project work",50 000 ₽ in hand,September 1998 — currently25 years 9 months
4,12 лет,Пермь,Мужчина,38 лет,Высшее образование,Пермский национальный исследовательский полите...,Русский — Родной Английский — B2 — Средне-прод...,Занятость: полная занятость,90 000 ₽ на руки,Сентябрь 2015 — по настоящее время8 лет 9 месяцев
...,...,...,...,...,...,...,...,...,...,...
395,9 лет,Барнаул,Женщина,32 года,Высшее образование,Алтайский государственный технический универси...,Русский — Родной Английский — B2 — Средне-прод...,"Занятость: полная занятость, частичная занятос...",35 000 ₽ на руки,Февраль 2016 — по настоящее время8 лет 4 месяца
396,1 год,Москва,Мужчина,29 лет,Неоконченное высшее образование,Московский государственный технический универс...,Русский — Родной,"Занятость: полная занятость, частичная занятос...",160 000 ₽ на руки,Март 2022 — Август 20226 месяцев
397,18 лет,Саратов,Мужчина,42 года,Высшее образование (Кандидат наук),Саратовский государственный университет,Русский — Родной Английский — B2 — Средне-прод...,Занятость: полная занятость,90 000 ₽ на руки,Сентябрь 2022 — Июнь 202310 месяцев
398,7 лет,Вологда,Мужчина,28 лет,Высшее образование (Магистр),Вологодский государственный технический универ...,Русский — Родной Английский — A2 — Элементарный,Занятость: полная занятость,120 000 ₽ на руки,Октябрь 2018 — по настоящее время5 лет 8 месяцев


In [14]:
employers_data_frame.to_csv('output.csv', sep='\t', encoding='utf-8')